# INF367 Mandatory 2

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt
from src import Model1, Model2, Model3, circuit1, circuit2, circuit3

In [ ]:
SEED = 367

## Data exploration and pre-processing

In [ ]:
X, y = load_iris(return_X_y=True)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=.7, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, train_size=.5, random_state=SEED)
print("Training size: ", len(X_train))
print("Validation size: ", len(X_val))
print("Test size: ", len(X_test))

In [ ]:
print("Features shape: ",X_train.shape)
print("Target shape: ",y_train.shape)
print(f"Feature value range: {np.min(X_train)} : {np.max(X_train)}")

In [ ]:
plot_histogram(Counter(y_train), title="Class distribution")

In [ ]:
scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)

## QNN-circuits

In [ ]:
_features = [0.4, 0.3, 0.1, 0.2] # Only for display purposes

In [ ]:
# Circuit 1
circ1 = circuit1(_features)
circ1.draw("mpl", reverse_bits=True, filename="images/circuit1.png")

In [ ]:
# Circuit 2
circ2 = circuit2(_features, layers=2)
circ2.draw("mpl", reverse_bits=True, filename="images/circuit2.png")

In [ ]:
# Circuit 3
circ3 = circuit3(_features, layers=2)
circ3.draw("mpl", reverse_bits=True, filename="images/circuit3.png")

## Training and Validation

In [ ]:
models = []
model_accuracies = []
n_samples = 5
lr_range = [1, 1.5] 
epsilon_range = [0.5, 1]
layers = [4, 8]
architectures = [Model1, Model2, Model3]

epochs = 20

np.random.seed(SEED)
for architecture in architectures:
    for i in range(n_samples):
        model_parameters = {
            "learning_rate": round(np.random.uniform(*lr_range), 3),
            "epsilon": round(np.random.uniform(*epsilon_range),3),
            "layers": np.random.randint(*layers)
        }
        print(f"Model architecture: {architecture}, sample: {i+1}")
        [print(f"{k}: {v}, ", end="") for k,v in model_parameters.items()]
        print()
        model = architecture(gradient_shots=1000, seed=SEED, **model_parameters)
        model = model.fit(epochs, X_train, y_train, X_val, y_val, patience=2)
        pred_train = model.predict(X_train)
        pred_val = model.predict(X_val)
        train_acc = accuracy_score(y_train, pred_train)
        val_acc = accuracy_score(y_val, pred_val)
        print(f"Training accuracy: {train_acc*100:.2f}%, Validation accuracy: {val_acc*100:.2f}%\n")
        
        models.append(model)
        model_accuracies.append(val_acc)

In [ ]:
selected_model = models[model_accuracies.index(max(model_accuracies))]

In [ ]:
def plot_loss(train_loss, val_loss):
    plt.title("Training and Validation loss")
    plt.plot(train_loss, label="Training")
    plt.plot(val_loss, label="Validation")
    plt.legend()
    plt.savefig("images/loss.png")
    plt.show()

In [ ]:
plot_loss(selected_model.train_loss, selected_model.val_loss)

## Test Performance

In [ ]:
preds = selected_model.predict(X_test)
test_accuracy = accuracy_score(y_test, preds)

In [ ]:
print(f"Test accuracy: {test_accuracy*100:.2f}%")
print(f"Model: {type(selected_model)},\
        learning_rate: {selected_model.learning_rate},\
        epsilon: {selected_model.epsilon},\
        layers: {selected_model.layers if type(selected_model) is not Model1 else ''}")

In [ ]:
cm = confusion_matrix(y_test, preds)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.savefig("images/confusion_matrix.png")
plt.show()